In [29]:
from bokeh.plotting import  figure, output_notebook, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.models.widgets import Select
from bokeh.palettes import Spectral4
from bokeh.transform import factor_cmap
from bokeh.layouts import column
from bokeh.models import TabPanel
import pandas as pd

# Ensure output_notebook() is called in the same cell
output_notebook()

# Load the dataset
df = pd.read_csv('data/clean1.csv')


df['Total Cereal Allocation'] = df[['Cereals allocated to other uses', 'Cereals allocated to animal feed', 'Cereals allocated to human food']].sum(axis=1)
df['Proportion Allocated to Human Food'] = df['Cereals allocated to human food'] / df['Total Cereal Allocation']




Loading BokehJS ...

In [30]:
# List of countries with small populations
small_population_countries = ["Vatican City","Antigua and Barbuda", "Monaco", "Gibraltar", "Tokelau", "Nauru", "Saint Barthelemy", "Tuvalu", "Macau", 
                              "Sint Maarten", "Saint Martin", "Bermuda", "San Marino", "Guernsey", "Anguilla", "Montserrat", 
                              "Jersey", "British Virgin Islands", "Liechtenstein", "Aruba", "Marshall Islands", 
                              "American Samoa", "Cook Islands", "Saint Pierre And Miquelon", "Saint Kitts And Nevis", "Niue",
                              "Cayman Islands", "Wallis And Futuna", "Maldives", "Malta", "Grenada", "United States Virgin Islands",
                              "Mayotte", "Saint Vincent And The Grenadines", "Barbados", "Antigua And Barbuda", "Curacao",
                              "Seychelles", "Palau", "Northern Mariana Islands","Africa (FAO)","Asia (FAO)", "Andorra", "Guam", "Isle Of Man", "Saint Lucia",
                              "Micronesia", "Singapore", "Tonga", "Dominica", "Bahrain", "Kiribati", "Turks And Caicos Islands",
                              "Sao Tome And Principe", "Hong Kong", "Martinique", "Faroe Islands", "Guadeloupe", "Comoros", 
                              "Mauritius", "Reunion", "Luxembourg", "Samoa", "Cape Verde", "French Polynesia", "Trinidad And Tobago",
                              "Brunei", "Palestine", "Puerto Rico", "Cyprus", "Lebanon", "Gambia", "Jamaica", "Qatar",
                              "Falkland Islands", "Vanuatu", "Montenegro", "Bahamas", "Timor Leste", "Eswatini", "Kuwait", "Fiji",
                              "New Caledonia", "Slovenia", "El Salvador", "Belize", "Djibouti", "North Macedonia", 
                              "Rwanda", "Burundi", "Equatorial Guinea", "Albania", "Solomon Islands", "Armenia",
                              "Lesotho", "Belgium", "Moldova", "Guinea Bissau"]

# Filter the DataFrame
df_large_population = df[~df['Entity'].isin(small_population_countries)]



In [31]:
# Remove rows with missing values in relevant columns
df_clean = df_large_population.dropna(subset=['Cereals allocated to other uses', 
                             'Cereals allocated to animal feed', 
                             'Cereals allocated to human food', 'Year', 'Entity'])

df_clean

Unnamed: 0       Entity Code  Year  Cereals allocated to other uses  \
0               0  Afghanistan  AFG  2014                           7000.0   
1               1  Afghanistan  AFG  2015                           5000.0   
2               2  Afghanistan  AFG  2016                           7000.0   
3               3  Afghanistan  AFG  2017                           7000.0   
4               4  Afghanistan  AFG  2018                           8000.0   
...           ...          ...  ...   ...                              ...   
12451       12451       Zambia  ZMB  2016                         158000.0   
12452       12452       Zambia  ZMB  2017                         180000.0   
12453       12453       Zambia  ZMB  2018                         108000.0   
12454       12454       Zambia  ZMB  2019                         169000.0   
12455       12455       Zambia  ZMB  2020                         340000.0   

       Cereals allocated to animal feed  Cereals allocated to human food  \
0                              704000.0                        6072000.0   
1                              748000.0                        6265000.0   
2                              472000.0                        6426000.0   
3                              403000.0                        6746000.0   
4                              410000.0                        6761000.0   
...                                 ...                              ...   
12451                          251000.0                        2398000.0   
12452                          266000.0                        2455000.0   
12453                          278000.0                        2427000.0   
12454                          290000.0                        2169000.0   
12455                          289000.0                        2605000.0   

       Needed habitable land area  Cropland  Pasture  Permanent ice  \
0                             NaN       NaN      NaN            NaN   
1                             NaN       NaN      NaN            NaN   
2                             NaN       NaN      NaN            NaN   
3                             NaN       NaN      NaN            NaN   
4                             NaN       NaN      NaN            NaN   
...                           ...       ...      ...            ...   
12451                         NaN       NaN      NaN            NaN   
12452                         NaN       NaN      NaN            NaN   
12453                         NaN       NaN      NaN            NaN   
12454                         NaN       NaN      NaN            NaN   
12455                         NaN       NaN      NaN            NaN   

       Semi-natural land  Urban  Villages  Wild barren land  Wild woodlands  \
0                    NaN    NaN       NaN               NaN             NaN   
1                    NaN    NaN       NaN               NaN             NaN   
2                    NaN    NaN       NaN               NaN             NaN   
3                    NaN    NaN       NaN               NaN             NaN   
4                    NaN    NaN       NaN               NaN             NaN   
...                  ...    ...       ...               ...             ...   
12451                NaN    NaN       NaN               NaN             NaN   
12452                NaN    NaN       NaN               NaN             NaN   
12453                NaN    NaN       NaN               NaN             NaN   
12454                NaN    NaN       NaN               NaN             NaN   
12455                NaN    NaN       NaN               NaN             NaN   

       Total Cereal Allocation  Proportion Allocated to Human Food  
0                    6783000.0                            0.895179  
1                    7018000.0                            0.892704  
2                    6905000.0                            0.930630  
3                    7156000.0                            0.942705  
4         

In [32]:
# Create a ColumnDataSource for each country
sources = {country: ColumnDataSource(df_clean[df_clean['Entity'] == country]) for country in countries}

# Create the initial figure
p = figure(width=800, height=250, x_axis_type="datetime", title=countries[0])
p.line(x='Year', y='Cereals allocated to human food', color=Spectral4[2], legend_label="Human Food", source=sources[countries[0]])

p.legend.location = "top_left"
p.legend.click_policy="hide"

hover = HoverTool(
    tooltips=[
        ("Year", "@Year"),
        ("Human Food", "@{Cereals allocated to human food}")
    ]
)
p.add_tools(hover)

# Create a Select widget for country selection
country_select = Select(value=countries[0], options=list(countries))

# Define a CustomJS callback for the Select widget
callback = CustomJS(args=dict(sources=sources, plot=p), code="""
    var country = cb_obj.value;
    plot.title.text = country;
    plot.renderers[0].data_source.data = sources[country].data;
    plot.change.emit();
""")
country_select.js_on_change('value', callback)

# Show the plot and the Select widget
show(column(country_select, p))


In [27]:
# Combine all figures in a tab
tabs = Tabs(tabs=figs)

# Show the plot
show(tabs)
